In [1]:
import os
import sys

os.environ["PYSPARK_PYTHON"] = '/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"] = '/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"] = '--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


# Создание сессии

In [6]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

sc = (
    SparkSession
    .builder
    .config(conf=conf)
    .appName("lab01-alexander.yusov")
    .getOrCreate()
)

# Данные

In [9]:
film_id = 328

u_data = sc.sparkContext.textFile("/labs/laba01/ml-100k/u.data")

## Парсинг tsv

In [14]:
# format | user_id | item_id | rating | timestamp

(
    u_data
    .map(lambda x: x.split("\t"))
    .top(3)
)

[['99', '98', '5', '885679596'],
 ['99', '978', '3', '885679382'],
 ['99', '975', '3', '885679472']]

# Задание

По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

Выходной формат файла — json. Пример решения:

```
{
   "hist_film": [  
      134,
      123,
      782,
      356,
      148
   ],
   "hist_all": [  
      134,
      123,
      782,
      356,
      148
   ]
}
```

В поле “hist_film” нужно указать для заданного id фильма количество поставленных оценок в следующем порядке: "1", "2", "3", "4", "5". То есть, сколько было единичек, двоек, троек и т.д.

В поле “hist_all” нужно указать то же самое, только для всех фильмов общее количество поставленных оценок в том же порядке: "1", "2", "3", "4", "5".

In [31]:
from operator import add

hist_film = (
    u_data
    .map(lambda x: x.split("\t"))  # парсинг строки
    .map(lambda x: (int(x[1]), int(x[2])))  # (item_id, rating)
    .filter(lambda x: x[0] == film_id)  # поиск нужного фильма
    .map(lambda x: (x[1], 1))  # (оценка, 1)
    .reduceByKey(add)  # типа word count
    .sortByKey()
    .take(6)  # на всякий случай, чтобы в случае ошибки в данных не было OOM
)
hist_film

[(1, 12), (2, 40), (3, 94), (4, 109), (5, 40)]

In [32]:
hist_all = (
    u_data
    .map(lambda x: x.split("\t"))  # парсинг строки
    .map(lambda x: (int(x[1]), int(x[2])))  # (item_id, rating)
    .map(lambda x: (x[1], 1))  # (оценка, 1)
    .reduceByKey(add)  # типа word count
    .sortByKey()
    .take(6)  # на всякий случай, чтобы в случае ошибки в данных не было OOM
)
hist_all

[(1, 6110), (2, 11370), (3, 27145), (4, 34174), (5, 21201)]

## Сохранение результата

In [36]:
import json

with open("/data/home/alexander.yusov/lab01.json", "w") as f:
    json.dump({
        "hist_film": [x[1] for x in hist_film],
        "hist_all": [x[1] for x in hist_all],
    }, f)

# Закрытие сессии

In [37]:
sc.stop()